In [46]:
import boto3

In [47]:
# SUBO MI CODIGO PYTHON EN UNA RUTA ESPECIFICA

BUCKET_NAME = 'attemp-glue'
AWS_DEFAULT_ACL = 'public-read'
REGION = 'us-west-1'
path_in_pc = '/home/rmg/STORAGE/tests_courses/practice/PYSPARK/GLUE/code_up.py'
path_s3 = 'code_up.py'

client = boto3.client('s3')
s3 = boto3.resource('s3')
bucket = s3.Bucket(BUCKET_NAME)

if not bucket.creation_date:
    s3.create_bucket(
        ACL=AWS_DEFAULT_ACL,
        Bucket=BUCKET_NAME,
        CreateBucketConfiguration={'LocationConstraint': REGION}
    )
    
client.upload_file(
    path_in_pc,
    BUCKET_NAME,
    path_s3,
    ExtraArgs={
        'ContentType': 'text/x-python',
        'ACL': AWS_DEFAULT_ACL
    }
)

In [29]:
# PARA EJECUTAR UNA SOLA VEZ
# Creando el job GLUE   
client_glue = boto3.client('glue')
response_glue = client_glue.create_job(
    Name='copy-csv-glue',
    Description='Prueba de copias csv en flujo GLUE',
    Role='AdminGlue',
    ExecutionProperty={
        'MaxConcurrentRuns': 3
    },
    Command={
        'Name': 'pythonshell',
        'ScriptLocation': 's3://attemp-glue/code_up.py'
    }
)
response_glue

{'Name': 'copy-csv-glue',
 'ResponseMetadata': {'RequestId': '76657c7d-4f49-11e9-9c9a-a7307ce5a5cd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 25 Mar 2019 22:00:54 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '24',
   'connection': 'keep-alive',
   'x-amzn-requestid': '76657c7d-4f49-11e9-9c9a-a7307ce5a5cd'},
  'RetryAttempts': 0}}

In [48]:
# RUN JOB GLUE
response_run = client_glue.start_job_run(
    JobName='copy-csv-glue'
)
response_run

{'JobRunId': 'jr_119116d17f7b4155b5b3017eb4bb0647173121000aaeba4ca6fba1e7cd90df4b',
 'ResponseMetadata': {'RequestId': '4327c914-5005-11e9-8c5e-7db0e252cd10',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 26 Mar 2019 20:25:14 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4327c914-5005-11e9-8c5e-7db0e252cd10'},
  'RetryAttempts': 0}}

In [49]:
job_run_id = response_run.get('JobRunId', '')
job_run_id

'jr_119116d17f7b4155b5b3017eb4bb0647173121000aaeba4ca6fba1e7cd90df4b'

In [53]:
response_status = client_glue.get_job_run(
    JobName='copy-csv-glue',
    RunId = job_run_id
)
response_status

{'JobRun': {'Id': 'jr_119116d17f7b4155b5b3017eb4bb0647173121000aaeba4ca6fba1e7cd90df4b',
  'Attempt': 0,
  'JobName': 'copy-csv-glue',
  'StartedOn': datetime.datetime(2019, 3, 26, 15, 25, 14, 481000, tzinfo=tzlocal()),
  'LastModifiedOn': datetime.datetime(2019, 3, 26, 15, 25, 38, 603000, tzinfo=tzlocal()),
  'CompletedOn': datetime.datetime(2019, 3, 26, 15, 25, 38, 603000, tzinfo=tzlocal()),
  'JobRunState': 'SUCCEEDED',
  'PredecessorRuns': [],
  'AllocatedCapacity': 1,
  'ExecutionTime': 12,
  'Timeout': 2880,
  'LogGroupName': '/aws-glue/python-jobs'},
 'ResponseMetadata': {'RequestId': '52ab7f1c-5005-11e9-bccb-f117eb004523',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 26 Mar 2019 20:25:40 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '382',
   'connection': 'keep-alive',
   'x-amzn-requestid': '52ab7f1c-5005-11e9-bccb-f117eb004523'},
  'RetryAttempts': 0}}